In [53]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

# Turn inFile into a one hot label
def createOneHot(inFile,amountOfClasses):
    outFile = np.zeros((inFile.shape[0],inFile.shape[1], amountOfClasses));
    for i in range(0,inFile.shape[0]):
        for j in range(0,inFile.shape[1]):
            #if inFile[i,j] != 0:
            outFile[i,j,int(inFile[i,j])] = 1;
    return outFile;

# Turn 41 orthomosaics into one array. From 1 path with Bands 1 to 41
def loadOneOrthomosaic(imagePath):
    temp= np.asarray( Image.open( imagePath + 'Band1.png') );
    inputData = np.zeros(( temp.shape[0], temp.shape[1], inputChannels),dtype=np.dtype('uint8'))
    inputData[:,:,0] = temp[:,:]
    for i in range(1,41):
        temp= np.asarray( Image.open( imagePath + 'Band' + str(i+1) + '.png') );
        inputData[:,:,i] = temp[:,:];
    return np.expand_dims( inputData, axis=0)

# Load one label from a .mat
def loadOneLabel(imagePath):
    temp = sio.loadmat(imagePath);
    trainOneHotLabels =  createOneHot( temp['labeledPicture'], amountOfClasses);
    return np.expand_dims( trainOneHotLabels, axis=0)

# Read training data and labels for an array of path tuples, returns (data,labels)
def readSet(paths):
    numberOfImages = len(paths);
    data = np.zeros( (numberOfImages,imageSize[1],imageSize[0],inputChannels), dtype=np.dtype('uint8'));
    labels = np.zeros((numberOfImages,imageSize[1],imageSize[0],amountOfClasses), dtype=np.dtype('uint8'))
    for i in range(0,numberOfImages):
        data[i,:,:,:] = loadOneOrthomosaic(paths[i][0]);
        labels[i,:,:,:] = loadOneLabel(paths[i][1]);
    return data,labels

def oneHotToBinary(oneHotLabel,threshold):
    temp = np.zeros((oneHotLabel.shape[1],oneHotLabel.shape[2]), dtype=np.dtype('uint8') );
    for i in range(0,oneHotLabel.shape[1]):
        for j in range(0,oneHotLabel.shape[2]):
            #temp[i,j] = np.argmax(oneHotLabel[0,i,j,:]) + 1;
            if np.argmax(oneHotLabel[0,i,j,:])>threshold:
                temp[i,j] = np.argmax(oneHotLabel[0,i,j,:])
    return temp

def predictAndSaveOneImage(inPath,outPath,FCNmodel,threshold):
    testImage = loadOneOrthomosaic(inPath);
    testLabelPredict = FCNmodel.predict(testImage);
    testLabelPredictBinary = oneHotToBinary(testLabelPredict,threshold);
    sio.savemat(outPath, mdict={'testLabelPredict': testLabelPredictBinary});

def predictAndSaveSet(pathArray, FCNmodel,threshold):
    numberOfImages = len(pathArray);
    for i in range(0,numberOfImages):
        predictAndSaveOneImage(pathArray[i][0], pathArray[i][1], FCNmodel,threshold)
        print('Saved ' + pathArray[i][1])
        
def plotImage(image):
    plt.imshow(image, cmap='gray')
    plt.show()
    
def getTrainPaths(paths):
    folders = ['FIP','Ximea_Tamron']
    fullPath = []
    for imageType in paths:
        for date in imageType:
            temp = []
            temp.append(basePath+'trainData/'+folders[paths.index(imageType)]+'/'+date+'/')
            temp.append(basePath+'trainLabels/'+folders[paths.index(imageType)]+'_'+date+'.mat') 
            fullPath.append(temp)
    return fullPath
    
def getTestPaths(paths):
    folders = ['FIP','Ximea_Tamron']
    fullPath = []
    for imageType in paths:
        for date in imageType:
            temp = []
            temp.append(basePath+'testData/'+folders[paths.index(imageType)]+'/'+date+'/')
            temp.append(basePath+'testLabelPredict/'+folders[paths.index(imageType)]+'_'+date+'.mat') 
            fullPath.append(temp)
    return fullPath

def getSampleWeight(label):
    sampleWeights = np.zeros((label.shape[0],label.shape[1]))
    counter = np.zeros(amountOfClasses)
    numberOfElements = label.shape[0]*label.shape[1]
    for i in range(0,label.shape[0]):
        for j in range(0,label.shape[1]):
            counter[ np.argmax(label[i,j]) ] += 1
    for i in range(0,label.shape[0]):
        for j in range(0,label.shape[1]):
            if counter[np.argmax(label[i,j])] != 0:
                sampleWeights[i,j] =  numberOfElements / counter[np.argmax(label[i,j])]
    return sampleWeights

def getSampleWeights(labels):
    sampleWeightArray = np.zeros((labels.shape[0],labels.shape[1],labels.shape[2]))
    for i in range(labels.shape[0]):
        sampleWeightArray[i,:,:] = getSampleWeight(labels[i,:,:,:])
    return sampleWeightArray



"""
A weighted version of categorical_crossentropy for keras (2.0.6). This lets you apply a weight to unbalanced classes.
@url: https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d
@author: wassname
"""
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [49]:
test = getSampleWeights(trainLabels)

In [50]:
test.shape

(3, 1600, 1600)

In [2]:
# Data paths
basePath = '/Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/';

trainSets = [
    #FIP
    [
        '20170622'
    ],
    #Ximea_Tamron
    [
        '20170510',
        '20170622'
    ]
]

validationSets = [
    #FIP
    [
        '20170531'
    ],
    #Ximea_Tamron
    [
    ]
]

testSets = [
    #FIP
    [
        '20170531', 
        '20170622',
        '20170531_cropped',
        '20170622_cropped',
        '20170802'
    ],
    #Ximea_Tamron
    [
        '20170510',
        '20170622',
        '20170510_cropped',
        '20170622_cropped',
        '20170613'
    ]
]
    
trainPaths = getTrainPaths(trainSets)
validationPaths = getTrainPaths(validationSets)
testPaths = getTestPaths(testSets)

# Data parameters
amountOfClasses = 9;
inputChannels = 41;
imageSize = [1600,1600]; # [X,Y]

numberOfEpochs = 20;

In [11]:
from keras.models import *
from keras.layers import Conv2D, MaxPooling2D, Input, Cropping2D, UpSampling2D,concatenate,Dropout

#crop1 = Cropping2D(cropping=((0, 0), (0, 0)))(conv1) # cropping=((Top, Bottom), (Left, Right))

model_inputs = Input(shape=(imageSize[0], imageSize[1], inputChannels))

conv1 = Conv2D(64,3, activation='relu', padding='same')(model_inputs)
conv1 = Conv2D(64,3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')(conv1)

conv2 = Conv2D(128,3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128,3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')(conv2)

conv3 = Conv2D(256,3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256,3, activation='relu', padding='same')(conv3)
drop3 = Dropout(0.5)(conv3)

upconv4 = Conv2D(128,2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop3))
concat4 = concatenate([upconv4,conv2],axis=3)
conv4 = Conv2D(128,3, activation='relu', padding='same')(concat4)
conv4 = Conv2D(128,3, activation='relu', padding='same')(conv4)

upconv5 = Conv2D(64,2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv4))
concat5 = concatenate([upconv5,conv1],axis=3)
conv5 = Conv2D(64,3, activation='relu', padding='same')(concat5)
conv5 = Conv2D(64,3, activation='relu', padding='same')(conv5)

conv6 = Conv2D(amountOfClasses, (1,1), activation='softmax', padding='valid')(conv5)


UNetModel = Model(inputs = model_inputs, outputs = conv6)
UNetModel.compile(optimizer='adagrad',
              loss='categorical_crossentropy',
              metrics=['accuracy']);
print(UNetModel.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1600, 1600, 4 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 1600, 1600, 6 23680       input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 1600, 1600, 6 36928       conv2d_18[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 800, 800, 64) 0           conv2d_19[0][0]                  
__________________________________________________________________________________________________
conv2d_20 

In [58]:
modelBasicNN = Sequential([
    Conv2D(20, (1,1), input_shape=(imageSize[0], imageSize[1], inputChannels), activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])
modelBasicNN.summary()

weights = np.array([1,1,1,1,1,1,1,1,1])*3
loss = weighted_categorical_crossentropy(weights)
#"categorical_crossentropy"
opt = keras.optimizers.SGD(lr=0.01)
modelBasicNN.compile(loss = loss, optimizer = opt,  metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 1600, 1600, 20)    840       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1600, 1600, 9)     189       
Total params: 1,029
Trainable params: 1,029
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Load the training data and labels
trainData,trainLabels = readSet(trainPaths)
validationData,validationLabels = readSet(validationPaths)

#trainWeights = getSampleWeights(trainLabels)

In [ ]:
# Train model
modelBasicNN.fit(trainData, trainLabels,validation_data=(validationData,validationLabels), epochs=10)

Train on 3 samples, validate on 1 samples
Epoch 1/10
3/3 [==============================] - 47s 16s/step - loss: 16.1822 - acc: 0.7528 - val_loss: 9.1205 - val_acc: 0.9135
Epoch 2/10
3/3 [==============================] - 48s 16s/step - loss: 16.0633 - acc: 0.7528 - val_loss: 9.1109 - val_acc: 0.9125
Epoch 3/10


In [112]:
# Save the model
UNetModel.save('networkModels/UNetModel.h5')

In [57]:
# Use trained model to predict
predictAndSaveSet(testPaths, modelBasicNN,.1)

Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170531.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170622.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170531_cropped.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170622_cropped.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170802.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/Ximea_Tamron_20170510.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/Ximea_Tamron_20170622.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/Ximea_Tamron_20170510_cropped.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/Ximea_Tamron_20170622_cropped.mat
Saved /Volumes/mac_jannic_2017/thanujan/Datasets/x

In [68]:
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy
import  tensorflow as tf

sess = tf.Session()

a=np.zeros((1600,1600))
a1=createOneHot(a,amountOfClasses)
b1=trainLabels[0,:,:,:];
print(categorical_accuracy(a1,b1))
a = tf.convert_to_tensor(a1, dtype=tf.float32)
b = tf.convert_to_tensor(b1, dtype=tf.float32)
print(a.shape)
print(b.shape)
losss = categorical_crossentropy(a,b)
print(losss)
#print(losss.eval(session=sess))

Tensor("Cast_7:0", shape=(1600, 1600), dtype=float32)
(1600, 1600, 9)
(1600, 1600, 9)
Tensor("Neg_13:0", shape=(1600, 1600), dtype=float32)


ValueError: GraphDef cannot be larger than 2GB.

In [44]:
Sum = losss.eval(session=sess);
print(sum(sum(Sum)))

0.30517578125


In [74]:
count=0;
for i in range(0,1600):
    for j in range(0,1600):
        if np.argmax(b1[i,j,:]) == 8: count +=1
print(count)

2248752


In [49]:
z = categorical_accuracy(a,b)
print(sum(sum(z.eval(session=sess))))

2560000.0


In [73]:
print(b1[0,0])

[0 0 0 0 0 0 0 0 1]


In [66]:
whos

Variable                   Type          Data/Info
--------------------------------------------------
Conv2D                     type          <class 'keras.layers.convolutional.Conv2D'>
Conv2DTranspose            type          <class 'keras.layers.conv<...>utional.Conv2DTranspose'>
Image                      module        <module 'PIL.Image' from <...>e-packages/PIL/Image.py'>
MaxPooling2D               type          <class 'keras.layers.pooling.MaxPooling2D'>
Sequential                 type          <class 'keras.models.Sequential'>
Sum                        ndarray       1600x1600: 2560000 elems, type `float32`, 10240000 bytes (9.765625 Mb)
a                          Tensor        Tensor("Const_42:0", shap<...> 1600, 9), dtype=float32)
a1                         ndarray       1600x1600x9: 23040000 elems, type `float64`, 184320000 bytes (175.78125 Mb)
amountOfClasses            int           9
b                          Tensor        Tensor("Const_43:0", shap<...> 1600, 9), dtype=fl

In [67]:
sess.close()

In [30]:
testPaths[1][1]

'/Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170622.mat'